In [18]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import json
import geoviews

# Import API key
from api_keys import geoapify_key

In [19]:
#set the city
target_city = "Melbourne, Australia"
city = "Melbourne"

# Build the latitude search URL
url = f"https://api.geoapify.com/v1/geocode/search?text={target_city}&format=json&apiKey={geoapify_key}"

# Run a request to endpoint and convert result to json
results = requests.get(url).json()

#create the co-ordinate variables
lat =results["results"][0]["lat"]
lon = results["results"][0]["lon"]

# print the results
print(target_city, lat, lon)

Melbourne, Australia -37.8142176 144.9631608


In [29]:
# set city to search for infomration against
cities = ["Melbourne"]

# creates lists to hold the required information results
school_name = []
school_address = []
school_postcode =[]
school_lat = []
school_lon = []
school_distance = []
# school_type = []
# school_grades = []


# set the search parameters
radius = 20000
categories = "education.school"

# set the parameters for the type of search
filters = f"circle:{lon},{lat},{radius}"
bias = f"proximity:{lon},{lat}"
limit = 20

# set the search URL
base_url = "https://api.geoapify.com/v2/places"

# create the parameters dictionairy
params = {"categories": categories, "apiKey": geoapify_key, "filter": filters, "limit": limit, "bias": bias}

# create loop to to search based on city lat & long and append to list
for city in cities:
    response = requests.get(base_url, params=params).json()
    num_features = len(response["features"])

    for i in range(min(num_features,20)):
        school_name.append(response["features"][i]["properties"]["address_line1"])
        school_address.append(response["features"][i]["properties"]["address_line2"])
        school_postcode.append(response["features"][i]["properties"]["postcode"])
        school_lat.append(response["features"][i]["geometry"]["coordinates"][1])
        school_lon.append(response["features"][i]["geometry"]["coordinates"][0])
        school_distance.append(response["features"][i]["properties"]["distance"])
#         school_type.append(response["features"][i]["properties"]["datasource"]["raw"]["operator:type"])
#         school_grades.append(response["features"][i]["properties"]["datasource"]["raw"]["grades"])


# for each result, load to dataframe
results_df = pd.DataFrame({
    "school name": school_name,
    "address": school_address,
    "postcode": school_postcode,  
    "latitude": school_lat,
    "longitude": school_lon,
    "distance from CBD": school_distance,
#     "school type": school_type,
#     "school grades": school_grades,
})

# show results
results_df.head()

,school name,address,postcode,latitude,longitude,distance from CBD
0,Ozford College,"310 King Street, Melbourne VIC 3000, Australia",3000,-37.812797,144.954404,787
1,Holmes Grammar School,"185 Spring Street, Melbourne VIC 3000, Australia",3000,-37.810321,144.972187,905
2,Haileybury College City Campus,"383 King Street, West Melbourne VIC 3003, Aust...",3003,-37.811119,144.952924,931
3,Simonds Catholic College (St Mary's 7–10 Campus),"273 Victoria Street, West Melbourne VIC 3051, ...",3051,-37.806108,144.952881,1201
4,River Nile School,"117 Capel Street, North Melbourne VIC 3050, Au...",3050,-37.805078,144.954952,1246


In [30]:
# Add an empty column for school type and school grades
results_df ["school type"] = ""
results_df ["school grades"] = ""
# Display sample data
results_df

,school name,address,postcode,latitude,longitude,distance from CBD,school type,school grades
0,Ozford College,"310 King Street, Melbourne VIC 3000, Australia",3000,-37.812797,144.954404,787,,
1,Holmes Grammar School,"185 Spring Street, Melbourne VIC 3000, Australia",3000,-37.810321,144.972187,905,,
2,Haileybury College City Campus,"383 King Street, West Melbourne VIC 3003, Aust...",3003,-37.811119,144.952924,931,,
3,Simonds Catholic College (St Mary's 7–10 Campus),"273 Victoria Street, West Melbourne VIC 3051, ...",3051,-37.806108,144.952881,1201,,
4,River Nile School,"117 Capel Street, North Melbourne VIC 3050, Au...",3050,-37.805078,144.954952,1246,,
5,St Joseph's Flexible Learning Centre,"385 Queensberry Street, North Melbourne VIC 30...",3050,-37.803974,144.954902,1306,,
6,Carlton Gardens Primary School,"20-231 Rathdowne Street, Carlton VIC 3053, Aus...",3053,-37.801943,144.969294,1429,,
7,Academy of Mary Immaculate,"88 Nicholson Street, Fitzroy VIC 3065, Australia",3065,-37.803655,144.974504,1478,,
8,Victorian College of the Arts Secondary School,"55-65 Miles Street, Southbank VIC 3006, Australia",3006,-37.828521,144.964938,1558,,
9,South Melbourne Primary School,"129 Ferrars Street, Southbank VIC 3006, Australia",3006,-37.829395,144.953789,1818,,


In [ ]:
# set city to search for infomration against
# for index, row in results_df.iterrows():
    
#     # Get the city's name
#     school = row["school name"]
    
#     # Get latitude, longitude from the DataFrame
#     latitude = row["school latitude"]
#     longitude = row["school longitude"]
    
#     # Add the current city's latitude and longitude to the params dictionary
#     params["filter"] = f"circle:{longitude},{latitude},{radius}"
#     params["bias"] = f"proximity:{longitude},{latitude}"
    
#     # Set base URL
#     new_url = "https://api.geoapify.com/v2/places"
           
#     # Make an API request using the params dictionary
#     new_response = requests.get(new_url, params=params)
        
#     # Convert the API response to JSON format
#     new_response = new_response.json()
    
#     # Grab the first airport from the results and store the details in the DataFrame
#     try:
#         results_df.loc[index, "school type"] = new_response["features"][0]["properties"]["datasource"]["raw"]["operator:type"]
#         results_df.loc[index, "school grades"] = new_response["features"][0]["properties"]["datasource"]["raw"]["grades"]
#     except IndexError:
#         results_df.loc[index, "school type"] = None
#         results_df.loc[index, "school grades"] = None        
# # Display sample data
# results_df.head()

In [47]:
%%capture --no-display

# Configure the map plot
school_plot = results_df.hvplot.points(
    "longitude",
    "latitude",
    geo = True,
    global_extent = False,
    tiles = "OSM",
    hover_cols = "all",
    frame_width = 700,
    frame_height = 500,
    size = 100,
    color = "school name",
)

# show the map plot
school_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [longitude,latitude]   (school name,index,address,postcode,distance from CBD,school type,school grades)